# Getting the data

In [1]:
import preprocessing
import featureSelection
from sklearn.model_selection import train_test_split

Selection based on Pearson Correlation


c:\users\aaate\appdata\local\programs\python\python38\lib\site-packages\numpy\lib\function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
c:\users\aaate\appdata\local\programs\python\python38\lib\site-packages\numpy\lib\function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


25 selected features
['URG Flag Count', 'Fwd IAT Mean', 'Flow IAT Mean', 'Destination Port', 'Fwd IAT Total', 'Flow Duration', 'PSH Flag Count', 'Flow IAT Std', 'Idle Min', 'Idle Mean', 'Flow IAT Max', 'Fwd IAT Max', 'Idle Max', 'Average Packet Size', 'Fwd IAT Std', 'Packet Length Mean', 'Packet Length Variance', 'Max Packet Length', 'Min Packet Length', 'Packet Length Std', 'Bwd Packet Length Min', 'Avg Bwd Segment Size', 'Bwd Packet Length Mean', 'Bwd Packet Length Max', 'Bwd Packet Length Std']
Selection based on chi square distibution
25 selected features
['Destination Port', 'Flow Duration', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow IAT Std', 'Flow IAT Max', 'Fwd IAT Total', 'Fwd IAT Std', 'Fwd IAT Max', 'Bwd IAT Std', 'Max Packet Length', 'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count', 'PSH Flag Count', 'ACK Flag Count', 'URG Flag Count', 'Average Packet Size', 'Avg Bwd Segment

In [50]:
corf = featureSelection.cor_feature[:23]
chif = featureSelection.chi_feature[:25]
# corfm = corf[:10] + corf[20:]
# print(len(corfm))

schi = set(corf)
scor = set(chif)

sint = schi.intersection(scor)
feature_list = list(sint)
print(len(feature_list))

20


In [51]:
X_train, y_train = featureSelection.X_train, featureSelection.y_train
X_train = X_train[feature_list]

In [52]:
X_cv, y_cv = featureSelection.X_cv, featureSelection.y_cv
X_cv = X_cv[feature_list]

In [53]:
X_train.info()
X_cv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 295264 entries, 0 to 295263
Data columns (total 20 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Fwd IAT Max             295264 non-null  float64
 1   Idle Max                295264 non-null  float64
 2   Destination Port        295264 non-null  float64
 3   Average Packet Size     295264 non-null  float64
 4   Fwd IAT Std             295264 non-null  float64
 5   Packet Length Std       295264 non-null  float64
 6   URG Flag Count          295264 non-null  float64
 7   Packet Length Mean      295264 non-null  float64
 8   Packet Length Variance  295264 non-null  float64
 9   Idle Min                295264 non-null  float64
 10  Bwd Packet Length Mean  295264 non-null  float64
 11  Flow IAT Max            295264 non-null  float64
 12  Flow Duration           295264 non-null  float64
 13  Flow IAT Std            295264 non-null  float64
 14  Idle Mean           

# Neural Network

In [58]:
import tensorflow as tf
import keras
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense
import random
from numpy.random import seed
import matplotlib.pyplot as plt
import numpy as np
from keras.optimizers import Adam


# X_train = X_train.loc[:65535]
# y_train = y_train[:65536]
model = Sequential()
model.add(Dense(2*len(feature_list), input_dim=len(feature_list), activation='relu'))
model.add(Dense(2*len(feature_list), activation='relu'))
model.add(Dense(len(feature_list), activation='relu'))
# model.add(Dense(2*len(feature_list), activation='relu'))
# model.add(Dense(len(feature_list), activation='relu'))
model.add(Dense(len(feature_list)//2, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.005), metrics=['accuracy'])

model.fit(X_train, y_train, epochs=100, batch_size=256)


Epoch 1/100
295264/295264 [==============================] - 5s 15us/step - loss: 0.1753 - accuracy: 0.9294
Epoch 2/100
295264/295264 [==============================] - 4s 15us/step - loss: 0.1240 - accuracy: 0.9542
Epoch 3/100
295264/295264 [==============================] - 4s 15us/step - loss: 0.1148 - accuracy: 0.9586
Epoch 4/100
295264/295264 [==============================] - 4s 15us/step - loss: 0.1027 - accuracy: 0.9638
Epoch 5/100
295264/295264 [==============================] - 4s 15us/step - loss: 0.0886 - accuracy: 0.9706
Epoch 6/100
295264/295264 [==============================] - 4s 15us/step - loss: 0.0918 - accuracy: 0.9695
Epoch 7/100
295264/295264 [==============================] - 4s 15us/step - loss: 0.0843 - accuracy: 0.9725
Epoch 8/100
295264/295264 [==============================] - 4s 15us/step - loss: 0.0901 - accuracy: 0.9703
Epoch 9/100
295264/295264 [==============================] - 4s 15us/step - loss: 0.0834 - accuracy: 0.9730
Epoch 10/100
295264/295264 [

295264/295264 [==============================] - 5s 15us/step - loss: 0.0642 - accuracy: 0.9788
Epoch 76/100
295264/295264 [==============================] - 4s 15us/step - loss: 0.0617 - accuracy: 0.9795
Epoch 77/100
295264/295264 [==============================] - 4s 15us/step - loss: 0.0617 - accuracy: 0.9794
Epoch 78/100
295264/295264 [==============================] - 4s 15us/step - loss: 0.0615 - accuracy: 0.9795
Epoch 79/100
295264/295264 [==============================] - 4s 15us/step - loss: 0.0631 - accuracy: 0.9788
Epoch 80/100
295264/295264 [==============================] - 5s 16us/step - loss: 0.0607 - accuracy: 0.9794
Epoch 81/100
295264/295264 [==============================] - 4s 15us/step - loss: 0.0605 - accuracy: 0.9797
Epoch 82/100
295264/295264 [==============================] - 4s 15us/step - loss: 0.0599 - accuracy: 0.9795
Epoch 83/100
295264/295264 [==============================] - 4s 15us/step - loss: 0.0599 - accuracy: 0.9796
Epoch 84/100
295264/295264 [====

In [71]:
def eval_metrics(y, y_predicted):
    from sklearn.metrics import accuracy_score , precision_score , recall_score , f1_score
    print(accuracy_score(y, y_predicted))
    print(precision_score(y, y_predicted))
    print(recall_score(y, y_predicted))
    print(f1_score(y, y_predicted))

In [72]:
print('Train metrics')
y_train_pred = model.predict_classes(X_train)
eval_metrics(y_train, y_train_pred)
print('CV metrics')
y_cv_pred = model.predict_classes(X_cv)
eval_metrics(y_cv, y_cv_pred)

Train metrics
0.9793472959791915
0.9711268421205015
0.9845240541254513
0.9777795592350745
CV metrics
0.9789322291060817
0.9699504232572257
0.9848640504075064
0.9773503474759145


# Ensemble Learning

In [69]:
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score , precision_score , recall_score , f1_score

In [62]:
rdf_c = RandomForestClassifier(n_estimators=300,random_state=0)
knn_clf = KNeighborsClassifier()
ExtraTreesClassifier = ExtraTreesClassifier()
gradient_boost = GradientBoostingClassifier()
ada_mod = AdaBoostClassifier(base_estimator= None)
bg_mod = BaggingClassifier(base_estimator= None, bootstrap_features=False)

In [63]:
vot_classifier = VotingClassifier(estimators=[
    ('bagging_classifier', bg_mod),
    ('rnd_classifier', rdf_c),
    ('ExtraTreesClassifier', ExtraTreesClassifier),
    ('KNN_classifier', knn_clf),
],voting='soft', n_jobs=-1)

vot_classifier.fit(X_train, y_train)

VotingClassifier(estimators=[('bagging_classifier', BaggingClassifier()),
                             ('rnd_classifier',
                              RandomForestClassifier(n_estimators=300,
                                                     random_state=0)),
                             ('ExtraTreesClassifier', ExtraTreesClassifier()),
                             ('KNN_classifier', KNeighborsClassifier())],
                 n_jobs=-1, voting='soft')

In [74]:
print('Train metrics')
y_train_pred = vot_classifier.predict(X_train)
eval_metrics(y_train, y_train_pred)
print('CV metrics')
y_cv_pred = vot_classifier.predict(X_cv)
eval_metrics(y_cv, y_cv_pred)

Train metrics
0.9977545518586757
0.9954406295438437
0.9997138160791336
0.9975726466937836
CV metrics
0.9957801239094702
0.9929805431560296
0.9979111019793165
0.9954397171599858


In [ ]:
for clf in (ExtraTreesClassifier, knn_clf, vot_classifier):
    clf.fit(X_train, y_train)
    y1 = clf.predict(X_train)
    print(clf.__class__.__name__, accuracy_score(y_train, y1))
    print(precision_score(y_train, y1))
    print(recall_score(y_train, y1))
    print(f1_score(y_train, y1))

In [ ]:
for clf in (ExtraTreesClassifier, knn_clf, vot_classifier):
    clf.fit(X_cv, y_cv)
    y1 = clf.predict(X_cv)
    print(clf.__class__.__name__, accuracy_score(y_cv, y1))
    print(precision_score(y_cv, y1))
    print(recall_score(y_cv, y1))
    print(f1_score(y_cv, y1))